<a href="https://colab.research.google.com/github/steffenvogler/LLM2GPS/blob/main/WIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
pip install --upgrade --quiet geopy google-api-python-client google-cloud-aiplatform langchain-core google-generativeai langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.3 MB/s eta 0:00:00


In [23]:
# # Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

**Getting started**

In [22]:
import getpass
import pathlib
import textwrap

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API Key")

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

Enter API Key··········


In [11]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


the project must be present under https://console.cloud.google.com/ using the same account as used in above authentication

In [13]:
# LLM model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.0-pro",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=5,
    verbose=True,
)

### Just testing

In [23]:
result = llm.invoke("Write 10 concise bullet points of a good sales agent in a call center")
to_markdown(result.content)

> - **Excellent communication skills:** Clear, articulate, and persuasive.
> - **Strong product knowledge:** In-depth understanding of products and services.
> - **Empathy and active listening:** Ability to understand customer needs and build rapport.
> - **Problem-solving abilities:** Identifying and resolving customer issues effectively.
> - **Negotiation skills:** Ability to negotiate mutually beneficial outcomes.
> - **Time management and organization:** Efficiently handling multiple calls and tasks.
> - **Positive attitude and enthusiasm:** Maintaining a positive demeanor and motivating customers.
> - **Data analysis and reporting:** Tracking and analyzing call metrics to improve performance.
> - **CRM proficiency:** Expertise in using customer relationship management systems.
> - **Continuous learning and development:** Eagerness to enhance skills and knowledge.

In [36]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

In [47]:
template = """Your job is to guess the geographic loaction based on the description and keywords.
% USER input
{user_input}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_input"], template=template)

# Holds my 'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """Given a location, provide a structured output that starts with the continent, then country, federal state, then administrative district or city (if applicable). Everything separated by a whitespace. Only provide a single line of output. Never output any additional text. If there are more than one places that match the description or if you are not sure, add an asterisk sign ad the end of the response.
% USER geo
{user_geo}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_geo"], template=template)

# Holds my 'json' chain
geo_chain = LLMChain(llm=llm, prompt=prompt_template)

overall_chain = SimpleSequentialChain(chains=[location_chain, geo_chain], verbose=False)

In [41]:
from geopy.geocoders import Nominatim
from geopy import distance
geolocator = Nominatim(user_agent="LLM2GPS")

In [83]:
query = "Spreewald" # @param ["Berlin", "Copenhagen Zoo", "Spreewald", "Gjoa Haven", "Mt. Cook", "Zugspitze", "Sandwich island"] {allow-input: true}

In [84]:
improved_query = str(query + " " + overall_chain.run(query))

location = geolocator.geocode(improved_query)
print("original query: {}; LLM-improved query: {}".format(query,improved_query))

i = 0
def response_processing(location, i):
  if (location is None):
    #print (i)
    if i == 4:
      return
    #print("No location found")
    if i == 3:
      improved_query = query
    elif i == 2:
      improved_query = str(overall_chain.invoke(query))
    else:
      improved_query = str(overall_chain.run(query) + " " + query)
    location = geolocator.geocode(improved_query)
    i = i + 1
    response_processing(location,i)
  else:
    # try:
    #   print(improved_query)
    # except:
    #   print(query)
    print("GPS-coordinates: ({},{})".format(location.latitude, location.longitude))
    print("Address according to geocoding service: {}".format(location.address))
    #print(location.raw['boundingbox'])
    #print(location.raw)
    xy0 = (location.raw['boundingbox'][0], location.raw['boundingbox'][2])
    xy1 = (location.raw['boundingbox'][1], location.raw['boundingbox'][3])
    print("Great circle distance in km: {}".format(round(distance.great_circle(xy0, xy1).km,2)))
    east_west_dist = haversine(location.raw['boundingbox'][0], location.raw['boundingbox'][2], location.raw['boundingbox'][0], location.raw['boundingbox'][3])
    north_south_dist = haversine(location.raw['boundingbox'][0], location.raw['boundingbox'][2], location.raw['boundingbox'][1], location.raw['boundingbox'][2])
    print("north_south_distance: {} and east-west-distance: {} (all in km)".format(north_south_dist, east_west_dist))

response_processing(location, i)

original query: Spreewald; LLM-improved query: Spreewald Europe Germany
GPS-coordinates: (51.9532976,13.884878363643075)
Address according to geocoding service: Spreewald, Dahme-Spreewald, Brandenburg, Deutschland
Great circle distance in km: 52.8
north_south_distance: 41.025568336377674 and east-west-distance: 33.376934661327994 (all in km)


In [55]:
import math

def haversine(lat1, lon1, lat2, lon2):
  """Calculates the great circle distance between two points on a sphere.

  Args:
    lat1: The latitude of the first point in degrees.
    lon1: The longitude of the first point in degrees.
    lat2: The latitude of the second point in degrees.
    lon2: The longitude of the second point in degrees.

  Returns:
    The great circle distance between the two points in kilometers.
  """

  R = 6371  # Earth's radius in kilometers

  dlat = math.radians(float(lat2) - float(lat1))
  dlon = math.radians(float(lon2) - float(lon1))
  a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(float(lat1))) * math.cos(math.radians(float(lat2))) * math.sin(dlon / 2) ** 2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
  d = R * c

  return d

In [85]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [56]:
# TODO: of 'type' = 'peak' a width/height calculation does not make sense (may be use national park instead?)
# TODO: check the 'type' provided by Nominatim geocoders
east_west_dist = haversine(location.raw['boundingbox'][0], location.raw['boundingbox'][2], location.raw['boundingbox'][0], location.raw['boundingbox'][3])
north_south_dist = haversine(location.raw['boundingbox'][0], location.raw['boundingbox'][2], location.raw['boundingbox'][1], location.raw['boundingbox'][2])
print("north_south_distance: {} and east-west-distance: {} (all in km)".format(north_south_dist, east_west_dist))

north_south_distance: 870.5505959686125 and east-west-distance: 691.9055110412605 (all in km)
